<a href="https://colab.research.google.com/github/leonardo-toffalini/szgepes25-hf/blob/main/szgepes_hf_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feladat
Adjunk pontos 95%-os megbízhatóságú konfidencia intervallumot n(=10,40,160) elemű minta alapján az exponenciális eloszlás paraméterére (segítség: használjuk a gamma eloszlást) és hasonlítsuk össze az órán látott aszimptotikus intervallummal szimulációk alapján (hogy viszonyul egymáshoz a hosszuk, illetve a "találati arányuk"?)

# Solution

Let $X_1, X_2, \ldots, X_n$ be independent and identically distributed random variables from an exponential distribution with parameter $\lambda$.

We know that:

If $X \sim \text{Exp}(\lambda)$, then the probability density function is: $f(x) = \lambda e^{-\lambda x}$, where $x > 0$

The sum of exponentially distributed random variables follows a gamma distribution:

If $X_1, X_2, \ldots, X_n \sim \text{Exp}(\lambda)$, then $S = \sum_{i=1}^{n} X_i \sim \text{Gamma}(n, \lambda)$

The probability density function of
$\text{Gamma}(n, \lambda)$ is:
$$f(s) = \frac{\lambda^n s^{n-1} e^{-\lambda s}}{\Gamma(n)}$, $s > 0$$


To derive a $(1-\alpha)$ confidence interval for $\lambda$, we need to find bounds $[L(S), U(S)]$ such that:

$$
P\left(L(S) \le \lambda \le U(S)\right) = 1 - \alpha
$$


Using the properties of the gamma distribution:

If $S \sim \text{Gamma}(n, \lambda)$, then $\lambda S \sim \text{Gamma}(n, 1)$

Let $g_{\alpha/2}$ and $g_{1-\alpha/2}$ be the $\alpha/2$ and $(1-\alpha/2)$ quantiles of the $\text{Gamma}(n, 1)$ distribution:

$P(\lambda S \leq g_{\alpha/2}) = \alpha/2$

$P(\lambda S \geq g_{1-\alpha/2}) = \alpha/2$

From this:

$P(g_{\alpha/2} \leq \lambda S \leq g_{1-\alpha/2}) = 1 - \alpha$

Rearranging for $\lambda$:

$P\left(\frac{g_{\alpha/2}}{S} \leq \lambda \leq \frac{g_{1-\alpha/2}}{S}\right) = 1 - \alpha$

Therefore, the confidence interval bounds are:

$L(S) = \frac{g_{\alpha/2}}{S}$

$U(S) = \frac{g_{1-\alpha/2}}{S}$

In [1]:
exact_ci_gamma <- function(sample, alpha = 0.05) {
  n <- length(sample)
  sum_x <- sum(sample)

  # The sum of exponentials with parameter λ follows Gamma(n, λ)
  # So sum_x ~ Gamma(n, λ)
  # For a fixed sum_x, the confidence interval for λ is derived from:
  # P(qgamma(alpha/2, n) < λ*sum_x < qgamma(1-alpha/2, n)) = 1-alpha

  lower <- qgamma(alpha/2, shape = n) / sum_x
  upper <- qgamma(1-alpha/2, shape = n) / sum_x

  return(c(lower = lower, upper = upper))
}


In [2]:
lambda_param <- 5
sample_sizes <- c(10, 40, 160)

samples <- lapply(sample_sizes, function(sample_size) { rexp(sample_size, rate = lambda_param)})

In [3]:
a = lapply(samples, function(sample) {
  ci <- exact_ci_gamma(sample)

  cat("n:", length(sample), "\n")
  cat("95% confidence interval for λ: [", ci[1], ",", ci[2], "]\n\n")
})

n: 10 
95% confidence interval for λ: [ 1.785285 , 6.360537 ]

n: 40 
95% confidence interval for λ: [ 3.709676 , 6.921005 ]

n: 160 
95% confidence interval for λ: [ 3.784972 , 5.162455 ]



In [4]:
calculate_conf <- function(alpha, m, sample_sizes) {
  nn = sample_sizes
  talal = rep(0, times=length(nn))

  a <- 5
  for (j in 1:length(nn)) {
    for (i in 1:m) {
      xdat = rexp(nn[j], a) # exponenciális eloszlású minta
      lambda_ml = 1 / mean(xdat)
      error1 = qnorm(1 - alpha/2) * lambda_ml / sqrt(nn[j])

      left1 <- lambda_ml - error1
      right1 <- lambda_ml + error1

      if (left1 < a && a < right1)
        talal[j] = talal[j]+1
    }
    cat("n:", nn[j], "\n")
    cat(100 * talal[j]/m, "% confidence interval for λ: [", left1, ",", right1, "]\n\n")
  }
  return(c((100 * talal / m), left1, right1))
}

m = 100000
alpha = 0.05
sample_sizes = c(10, 40, 160)
temp = calculate_conf(alpha, m, sample_sizes)

n: 10 
95.461 % confidence interval for λ: [ 2.242508 , 9.553802 ]

n: 40 
95.209 % confidence interval for λ: [ 2.619744 , 4.97259 ]

n: 160 
94.969 % confidence interval for λ: [ 4.290923 , 5.864492 ]



### Conlclusions:
It looks as tough the exact confidence interval for smaller sample size deviates from the asymptotic result. However for larger sample size, the asymptotic results approach the analytical confidence interval.